In [2]:
%load_ext autoreload
%autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
from mirisim.config_parser import SimulatorConfig
from mirisim import MiriSimulation
import tso_lrs_sims_setup
from pipetest_tso_lrs_sim import star_scene, lrstso_sim_config

from importlib import reload

In this notebook I'm going to generate simulated MIRI time series LRS (slitless) data, for data management & pipeline testing purposes. To install Mirisim, see the [the public release webpage](http://miri.ster.kuleuven.be/bin/view/Public/MIRISim_Public). The target for the mock observations is WASP-62, an exoplanet host star with the following properties from [the exoplanet encyclopaedia](http://exoplanet.eu/catalog/wasp-62_b/):

* spectral type F7
* T_bb = 6230 K
* K = 8.994

K magnitude of 8.994 corresponds to a flux of 20e3 microJy at 8 micron (value obtained from the ETC), and I'll use this value for the source generation.

Using the ETC, I calculated the following number of groups for a high-SNR but unsaturated image:
ngrps = 100

Three simulations will be executed, for a short and an intermediate-duration exposure. The number of groups will be the same for all 3, as will the number of exposures (nexp = 1). The numbers of integrations will be 10 and 20 (these can be changed very easily).

### Steps in setting up the simulation

This notebook will go through the following steps:

* Create the scene
* Set up the simulation
* Run the simulation

Each step has its own function. Steps 1 and 2 will each write out a .ini file, which will be used as input for the final step.

In [4]:
arr = ['SLITLESSPRISM'] * 2
ngrp = [100] * 2
nints = [10, 20]


## Step 1: Creating the input scene (star_scene)

Here we'll create the input scene for the simulations using the function star_scene(). Arguments:

* scene_file: the filename for the .ini file
* write_cube: write the scene image out to a FITS file (optional; default=False)

The function returns a mirisim.skysim.scenes.CompositeSkyScene object.


In [5]:
scene_ini = star_scene(scene_file='lrstso_star_scene.ini', write_cube=False)

2021-11-15 15:26:02,050 - INFO - Initializing Point
2021-11-15 15:26:02,051 - INFO - Initializing Background


Creating scene. Scene will be written to file lrstso_star_scene.ini
File lrstso_star_scene.ini already exists. Will be overwritten.
lrstso_star_scene.ini exists, overwrite (y/[n])?y


In [6]:
print(scene_ini)

lrstso_star_scene.ini


## Step 2: Configuring the simulation

Now I'll set up the simulations and prepare to run them. Before we write out a new simconfig file, we delete any existing files in the directory. I'll set it up to loop through the different simulations. For this I wrote the function lrstso_sim_config. Check the docstring for descriptions and default values of the arguments. 

The function will write out another .ini file containing the simulation configuration, and it returns the output filename for further use.

In [7]:
old_files = glob.glob('*_simconfig.ini')

for of in old_files:
    os.remove(of)

for (a, g, i) in zip(arr, ngrp, nints):
    sim_ini = lrstso_sim_config(mode='lrs', arr=a, ngrp=g, nint=i, nexp=1, 
                   scene_file=scene_ini, out=True)
    print(sim_ini)

Found scene file lrstso_star_scene.ini
**** Simulation will use the WebbPSF model ****
lrstso_SLITLESSPRISM_100G10I1E_simconfig.ini
Found scene file lrstso_star_scene.ini
**** Simulation will use the WebbPSF model ****
lrstso_SLITLESSPRISM_100G20I1E_simconfig.ini


### Step 3: Run the simulation

In the following step we'll run the simulations for the 2 different cases. For each run, we need 3 input files: the scene, the simulation configuration, and the simulator setup file. The first and last of these remain the same for each run, and we loop through the list of 6 simulation config files.

After the simulation has run, the code renames the output directory to include the simulation settings to the directory.


In [8]:
cfg_files = glob.glob('*_simconfig.ini')

# configure the simulator engine - this requires no editing from the default
simulator_config = SimulatorConfig.makeSimulator(take_webbPsf=True)

for f in cfg_files[:1]:
    print(f)
    tmp = f.split('.')
    fcomps = tmp[0].split('_')
    sim = MiriSimulation.from_configfiles(f)
    sim.run()
    outdir = sorted(glob.glob('*_*_mirisim'), key=os.path.getmtime )[-1]
    new_outdir = 'pipetest_lrstso_{0}_{1}_{2}'.format(fcomps[1], fcomps[2], outdir)
    os.rename(outdir, new_outdir)
    print(outdir, new_outdir)


2021-11-15 15:26:46,084 - INFO - Using simulation configuration: lrstso_SLITLESSPRISM_100G10I1E_simconfig.ini
2021-11-15 15:26:46,086 - INFO - Using scene configuration: lrstso_star_scene.ini
2021-11-15 15:26:46,089 - INFO - MIRISim version: 2.4.1
2021-11-15 15:26:46,089 - INFO - MIRI Simulation started.
2021-11-15 15:26:46,090 - INFO - Output will be saved to: 20211115_152646_mirisim
2021-11-15 15:26:46,091 - INFO - Storing configs in output directory.


lrstso_SLITLESSPRISM_100G10I1E_simconfig.ini


2021-11-15 15:26:46,628 - INFO - Reading cosmic ray properties from parameter file /Users/skendrew/anaconda3/envs/miricle/lib/python3.8/site-packages/miri/simulators/scasim/cosmic_ray_properties.py
2021-11-15 15:26:46,650 - INFO - Reading detector properties from parameter file /Users/skendrew/anaconda3/envs/miricle/lib/python3.8/site-packages/miri/simulators/scasim/detector_properties.py
2021-11-15 15:26:46,691 - INFO - Storing dither pattern in output directory.
2021-11-15 15:26:46,693 - INFO - Using $CDP_DIR for location of CDP files: /Users/skendrew//CDP2.4.1
2021-11-15 15:26:46,693 - INFO - Setting up simulated Observation, with following settings:
2021-11-15 15:26:46,694 - INFO - Configuration Path: LRS_SLITLESS
2021-11-15 15:26:46,694 - INFO - Primary optical path: IMA
2021-11-15 15:26:46,695 - INFO - IMA Filter: P750L
2021-11-15 15:26:46,696 - INFO - IMA Subarray: SLITLESSPRISM
2021-11-15 15:26:46,696 - INFO - IMA detector readout mode: FAST
2021-11-15 15:26:46,697 - INFO - IMA

2021-11-15 15:31:09,764 - INFO -  Retrieving CDP file
  from sftp 'www.miricle.org/.../MIRI_FM_MIRIMAGE_P750L_SLITLESSPRISM_PIXELFLAT_07.01.01.fits'
  to cache '/Users/skendrew//CDP2.4.1/MIRI_FM_MIRIMAGE_P750L_SLITLESSPRISM_PIXELFLAT_07.01.01.fits' ...
2021-11-15 15:31:11,336 - INFO - Reading 'PIXELFLAT' model from '/Users/skendrew//CDP2.4.1/MIRI_FM_MIRIMAGE_P750L_SLITLESSPRISM_PIXELFLAT_07.01.01.fits'
2021-11-15 15:31:11,389 - INFO - *NOTE*: Could not find exact match for pixel flat-field for detector MIRIMAGE with FAST mode with filter='P750L'. An alternative (MIRI_FM_MIRIMAGE_P750L_SLITLESSPRISM_PIXELFLAT_07.01.01.fits) is being used.
2021-11-15 15:31:11,406 - INFO - Reading 'LINEARITY' model from '/Users/skendrew//CDP2.4.1/MIRI_FM_MIRIMAGE_LINEARITY_06.02.00.fits'
2021-11-15 15:31:11,615 - INFO - Reading 'READNOISE' model from '/Users/skendrew//CDP2.4.1/MIRI_FM_MIRIMAGE_FAST_READNOISE_07.01.00.fits'
2021-11-15 15:31:11,668 - INFO - Creating exposure_data with 520 rows x 72 columns 

20211115_152646_mirisim pipetest_lrstso_SLITLESSPRISM_100G10I1E_20211115_152646_mirisim
